# <center>Đồ án 3: Linear Regression</center>

# Thông tin sinh viên

- Họ và tên: 
- MSSV: 
- Lớp: 

# Import

In [814]:
import pandas as pd
import numpy as np

# Import thêm dữ thư viện nếu cần

# Đọc dữ liệu

In [815]:
# Đọc dữ liệu bằng pandas
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

# Lấy các đặc trưng X và giá trị mục tiêu y cho các tập huấn luyện (train) và kiểm tra (test)
X_train = train.iloc[:, :-1]    # Dataframe (chứa các đặc trưng huấn luyện)
y_train = train.iloc[:, -1]     # Series    (chứa 1 giá trị mục tiêu kiểm tra)

X_test = test.iloc[:, :-1]      # Dataframe (chứa các đặc trưng kiểm tra)
y_test = test.iloc[:, -1]       # Series    (chứa 1 giá trị mục tiêu kiểm tra)

# Sinh viên có thể sử dụng các khác nếu cần

- Chuyển dữ liệu thành `numpy array`:

In [816]:
X_train_np = X_train.to_numpy()
y_train_np = y_train.to_numpy()

X_test_np = X_test.to_numpy()
y_test_np = y_test.to_numpy()

# Cài đặt hàm

In [817]:
# Cài đặt các hàm cần thiết ở đây

# Tìm giá trọng số của w
def find_weight(X, y):
    return np.round(np.linalg.inv(X.T @ X) @ X.T @ y, 3)

# Tính giá trị dự đoán của y
def calc_y_predict(X, w):
    return np.sum(X * w.T, axis = 1)
    
# Tính độ lỗi MAE giữa các giá trị dự đoán và giá trị thực tế
def MAE(y_predict, y_real):
    return np.mean(np.abs(y_predict - y_real))

- Tìm mô hình tốt nhất:

In [818]:
# Tìm mô hình tốt nhất 
# table là dataframe
def find_best_model(table, feature, clusters):
    # Lấy thêm cột 'Salary' để giữ giá trị y khi xáo trộn
    feature.append("Salary")

    # Lấy bảng dữ liệu sau khi được xáo trộn các dòng
    table_np = table.loc[:, feature].sample(frac = 1).to_numpy()
    
    # Xóa 'Salary' ra khỏi các đặc trưng
    feature.remove("Salary")

    # Chia bảng dữ liệu thành các nhóm
    table_cluster = np.array_split(table_np, clusters)

    # Tìm min MAE trung bình của mỗi nhóm
    MAE_sum = np.zeros(len(feature))

    for c in range(clusters):
        for f in range(len(feature)):
            w_temporary = find_weight(table_cluster[c][:, f, None], table_cluster[c][:, -1, None])

            for i in range(clusters):
                if (c != i):
                    MAE_sum[f] += MAE(calc_y_predict(table_cluster[i][:, f, None], w_temporary), table_cluster[i][:, -1])

    # Do tất cả đều chia cho cùng 1 giá trị để tìm trung bình đặt là k
    # Nếu a/k > b/k thì a > b (với k > 0)
    return np.argmin(MAE_sum)

# Yêu cầu 1a: Sử dụng toàn bộ 11 đặc trưng đầu tiên `Gender`, `10percentage`, `12percentage`, `CollegeTier`, `Degree`, `collegeGPA`, `CollegeCityTier`, `English`, `Logical`, `Quant`, `Domain` (2 điểm) 

In [819]:
# Phần code cho yêu cầu 1a
X_train_1a = X_train_np[:, :11]
w = find_weight(X_train_1a, y_train_np)

In [820]:
# Gọi hàm MAE (tự cài đặt hoặc từ thư viện) trên tập kiểm tra
X_test_1a = X_test_np[:, :11]
y_predict = calc_y_predict(X_test_1a, w)
print("MAE:", MAE(y_predict, y_test))

MAE: 104863.71390646267


Công thức hồi quy (phần trọng số làm tròn đến 3 chữ số thập phân, ví dụ 0.012345 $\to$ 0.012)


\begin{aligned}
\text{Salary} = & -22756.513 \times Gender &+& 804.503 \times 10percentage &+& 1294.655 \times 12percentage \\
                & + -91781.898 \times CollegeTier &+& 23182.389 \times Degree &+& 1437.549 \times collegeGPA \\
                & + -8570.662 \times CollegeCityTier &+& 147.858 \times English &+& 152.888 \times Logical \\
                & + 117.222 \times Quant &+& 34552.286 \times Domain
\end{aligned}

# Yêu cầu 1b: Xây dựng mô hình sử dụng duy nhất 1 đặc trưng tính cách với các đặc trưng tính cách gồm `conscientiousness`, `agreeableness`, `extraversion`, `nueroticism`, `openess_to_experience`, tìm mô hình cho kết quả tốt nhất (1 điểm)

Lưu ý: khi sử dụng cross-validation, sinh viên cần xáo trộn dữ liệu 1 lần duy nhất và thực hiện trên toàn bộ đặc trưng

In [821]:
# Phần code cho yêu cầu 1b
# Tìm ra đặc trưng tốt nhất
# In ra các kết quả cross-validation như yêu cầu

feature = ["conscientiousness", "agreeableness", "extraversion", "nueroticism", "openess_to_experience"]
best_personality_feature_index = find_best_model(train, feature, len(feature))
print("Best feature: ", feature[best_personality_feature_index])

Best feature:  nueroticism


In [822]:
# Huấn luyện lại mô hình best_personality_feature_model với đặc trưng tốt nhất trên toàn bộ tập huấn luyện

def best_personality_feature_model(best_personality_feature_index):
    X_train_1b = X_train_np[:, 24 - best_personality_feature_index, None]
    return find_weight(X_train_1b, y_train_np)

In [823]:
# Gọi hàm MAE (tự cài đặt hoặc từ thư viện) trên tập kiểm tra với mô hình best_personality_feature_model
print("Index key [0-4]:", best_personality_feature_index)

w_1b = best_personality_feature_model(best_personality_feature_index)
print("Weight:", w_1b)

y_predict = np.sum(X_test_np[:, 24 - best_personality_feature_index, None] * w_1b.T, axis = 1)
print("MAE:", MAE(y_predict, y_test_np))

Index key [0-4]: 3
Weight: [-56546.304]
MAE: 291019.6931941854


Công thức hồi quy (phần trọng số làm tròn đến 3 chữ số thập phân, ví dụ 0.012345 $\to$ 0.012)

$$\text{Salary} = ...$$

# Yêu cầu 1c: Xây dựng mô hình sử dụng duy nhất 1 đặc trưng `English`, `Logical`, `Quant`, tìm mô hình cho kết quả tốt nhất (1 điểm)

Lưu ý: khi sử dụng cross-validation, sinh viên cần xáo trộn dữ liệu 1 lần duy nhất và thực hiện trên toàn bộ đặc trưng

In [824]:
# Phần code cho yêu cầu 1c
# Tìm ra đặc trưng tốt nhất
# In ra các kết quả cross-validation như yêu cầu

In [825]:
# Huấn luyện lại mô hình best_skill_feature_model với đặc trưng tốt nhất trên toàn bộ tập huấn luyện

In [826]:
# Gọi hàm MAE (tự cài đặt hoặc từ thư viện) trên tập kiểm tra với mô hình best_skill_feature_model


Công thức hồi quy (phần trọng số làm tròn đến 3 chữ số thập phân, ví dụ 0.012345 $\to$ 0.012)

$$\text{Salary} = ...$$

# Yêu cầu 1d: Sinh viên tự xây dựng mô hình, tìm mô hình cho kết quả tốt nhất (3 điểm)

Lưu ý: khi sử dụng cross-validation, sinh viên cần xáo trộn dữ liệu 1 lần duy nhất và thực hiện trên toàn bộ $m$ mô hình mà sinh viên thiết kế

## Tìm mô hình

In [827]:
# Trình bày các phần tìm ra mô hình

## Thử nghiệm, so sánh các mô hình

In [828]:
# Phần code cho yêu cầu 1d
# Tìm ra mô hình tốt nhất (tự thiết kế bởi sinh viên)
# In ra các kết quả cross-validation như yêu cầu

flag = np.array([0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1])

clusters = 10
train_table_suffle = train.iloc[:, flag].sample(frac = 1).to_numpy()
clusters_size = int(len(train_table_suffle) / clusters)


suffle_arr = np.array_split(train_table_suffle, clusters)


sum_MAE = np.zeros(5)

for i in range(clusters):
    for k in range(5):
        weight_temporary = find_weight(suffle_arr[i][:, k, None], suffle_arr[i][:, -1, None])

        for j in range(clusters):
            if (i != j):
                sum_MAE[k] += MAE(suffle_arr[j][:, k] * weight_temporary, suffle_arr[j][:, -1])

best_personality_feature_index = np.argmin(sum_MAE / (clusters - 1))

In [829]:
# Huấn luyện lại mô hình my_best_model trên toàn bộ tập huấn luyện


def my_best_model():
    flag = np.array([0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1], dtype=bool)
    X_train_1c = X_train_np[:, flag]
    return find_weight(np.square(X_train_1c), y_train_np)

In [830]:
# Gọi hàm MAE (tự cài đặt hoặc từ thư viện) trên tập kiểm tra với mô hình my_best_model
w = my_best_model()
flag = np.array([0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1], dtype=bool)
y_predict = np.sum(np.square(X_test_np[:, flag]) * w.T, axis=1)
MAE(y_predict, y_test_np)

98658.41379751194

Công thức hồi quy (phần trọng số làm tròn đến 3 chữ số thập phân, ví dụ 0.012345 $\to$ 0.012)

$$\text{Salary} = ...$$